# Setup 

To control matlab via Python, you'll need to have installed the MATLAB engine for Python, which is available since `Matlab 2019a`. On kinsler, this is done via:


```bash
module load Matlab/2019a
cd /apps/software/Matlab/R2019a/extern/engines/python/
python setup.py build --build-base=$(mktemp -d) install
```


👉 [Click here for general install instructions](https://uk.mathworks.com/help/matlab/matlab_external/install-the-matlab-engine-for-python.html)

To visualize 3D volumes (only via Google Chrome), install [k3d-jupyter](https://github.com/K3D-tools/K3D-jupyter)


```bash
pip install k3d
```

# Calling MATLAB and kWave functions

In [2]:
import jwave
from jwave.geometry import kGrid, Medium
from jwave.matlab import Matlab
from jwave.physics import solve_helmholtz
from jax import numpy as jnp
import numpy as np
import k3d

## Opening a Matlab Instance

The MATLAB engine can be started via the `start()` method of `jwave.matlab.Matlab`. It requires the paths for `k-Wave` and `off-grid-sources`. There are two ways to pass such information. 

The most direct method gives them as arguments to the construction of the `Matlab` object. 

```python
mlb=jwave.matlab.Matlab(kwave_path="kwave/path", offgrid_path="offgrid/path2")
```

However, it is often tedious to write the paths every time a MATLAB instance is needed. Because the location of the `k-Wave` packages depends from the system, setting the paths by hand also makes notebooks and scripts non portable between different machines.

Alternatively, if no paths are explicitly given during the constructor call, the method automatically looks for the OS environment variables `KWAVE_CORE_PATH` and `KWAVE_OFFGRID_PATH`. Such variables can be manually set each time a new shell/bash is opened, or automatically set by appending the following lines to the `.bashrc` file:
```bash
KWAVE_CORE_PATH="/home/antonio/repos/k-wave-matlab/k-Wave"
KWAVE_OFFGRID_PATH="/home/antonio/repos/off-grid-sources"
export KWAVE_CORE_PATH
export KWAVE_OFFGRID_PATH
```
Alternatively, they can be directly set in notebook/scripts using the `os` package by running the following cell

In [3]:
import os
os.environ["KWAVE_CORE_PATH"]="/mnt/Software/k-Wave"
os.environ["KWAVE_OFFGRID_PATH"]="/mnt/Software/off-grid-sources"

In [4]:
mlb = Matlab()

Note that you can join opened MATLAB sessions using `mlb.start(join_session=True)`. This hooks python to the matlab engine of a currently open istance, which provides additional feedback trough the MATLAB desktop interface. 

⚠️ **This doesn't work in remote sessions**, in that case a new MATLAB session will be opened

In [5]:
mlb.start()

Opening new matlab session


## Running MATLAB commands from python

In [6]:
### Settings
ppw = 4.     #number of points per wavelength at c0
pml_size = 20

# source parameters
source_f0       = 500e3    # source frequency [Hz]
source_mag      = 60e3     # source pressure [Pa]
bowl_roc        = 64e-3    # bowl radius of curvature [m]
bowl_diameter   = 64e-3    # bowl aperture diameter [m]

# grid parameters
axial_size      = 120e-3   # total grid size in the axial dimension [m]
lateral_size    = 70e-3    # total grid size in the lateral dimension [m]
source_x_offset = 41
source_y_offset = 41

# off grid source parameters
bli_tolerance   = 0.02
upsampling_rate = 10

# medium properties
c0 = 1500;       # reference sound speed used for calculations
rho0 = 1000;     # reference density used for calculations
     
# calculate the grid spacing based on PPW and F0
dx = c0 / (ppw * source_f0);   # [m]

To add variables to the MATLAB workspace, use the `add()` method, specifying value and the variable name to be used in the MATLAB workspace

In [7]:
mlb.add(axial_size, "axial_size")
mlb.add(lateral_size, "lateral_size")
mlb.add(dx, "dx")

Getting outputs from matlab functions can be done via the `run()` function, using the argument `nargout=1`

In [8]:
raw_Nx = mlb.run("roundEven(axial_size / dx)", nargout=1) # Note the `nargout=1`

Nx = int(raw_Nx) + 1 + source_x_offset
Ny = int(mlb.run("roundEven(lateral_size / dx)", nargout=1)) + 1 + source_x_offset

You can also save the output of MATLAB commands by specifying a variable name

In [9]:
mlb.add(Nx, "Nx")
mlb.add(Ny, "Ny")

# The following saves `double(Nx)` back into the `Nx` variable
mlb.run("double(Nx)",to="Nx")

mlb.run("double(Ny)",to="Ny")
mlb.run("kWaveGrid(Nx, dx, Ny, dx, Ny, dx)", "kgrid") # Creates a new kWaveGrid in the variable kgrid

In [10]:
kgrid = kGrid.make_grid([Nx, Ny, Ny], [dx]*3) # Creates the corresponding jWave kGrid object

In [11]:
# Define source
mlb.add(bli_tolerance, "bli_tolerance")
mlb.add(upsampling_rate, "upsampling_rate")
mlb.run("double(upsampling_rate)","upsampling_rate")
mlb.run(
    "kWaveArray('BLITolerance', bli_tolerance, 'UpsamplingRate', upsampling_rate);", 
    "karray"
)
                
# set source position and orientation
source_pos = [float(kgrid.space_axis[0]._value[0]) + source_x_offset * kgrid.dx[0], 0, 0]
focus_pos = [float(kgrid.space_axis[0]._value[-1]), 0, 0]

#add bowl shaped element
mlb.add(source_pos, "source_pos")
mlb.add(focus_pos, "focus_pos")
mlb.add(bowl_roc, "bowl_roc")
mlb.add(bowl_diameter, "bowl_diameter")

mlb.run("karray.addBowlElement(source_pos, bowl_roc, bowl_diameter, focus_pos)")

In [12]:
# Get mask and source field (takes a bit)
src_mask = mlb.run("karray.getArrayBinaryMask(kgrid);", nargout=1)
src_field = mlb.run("karray.getArrayGridWeights(kgrid);", nargout=1)

# Run the simulation in jWave

In [13]:
medium = Medium(
    sound_speed=1500.*jnp.ones((Nx,Ny,Ny)),
    density=1000,
    attenuation=0.0,
    pml_size=10
)

src_field = np.array(src_field).astype(np.complex64)

omega = 2*np.pi*source_f0

In [14]:
field = solve_helmholtz(
    kgrid, 
    medium, 
    src_field, 
    omega,
    maxiter=10000
).block_until_ready()

The following cell visualizes a volume rendering of the results. This won't be visible in the documention, but only via running the Jupyter Notebook file.

In [15]:
vol = np.abs(field)
vol[vol>4000]=4000

volume = k3d.volume(
    vol,
    alpha_coef=30,
    shadow='dynamic',
    samples=600,
    shadow_res=128,
    shadow_delay=50,
    color_range=[300,np.amax(vol)]
)

src = np.abs(src_field)

src_volume = k3d.volume(
    src,
    alpha_coef=1000,
    shadow='dynamic',
    samples=600,
    shadow_res=128,
    shadow_delay=50,
    color_range=[0.1,10],
    color_map=k3d.colormaps.matplotlib_color_maps.Gist_gray
)

plot = k3d.plot(camera_auto_fit=False, height=1000)
plot += src_volume
plot += volume
plot.lighting = 2
plot.display()

Output()